In [1]:
import numpy as np
import matplotlib.pyplot as plt

## SLIP  
library to calculate Spring Loaded Inverted Pendulum

In [ ]:
# defined params
RIGHT = 0
LEFT = 1

In [2]:
# mechanical params
g = 9.81 # m/s^2
m = 1.2 # kg

In [ ]:
# robot initial settings
z0 = 158 * 0.001
foot_dist_y_base = 0.03
foot_dist_x_max = 0.12

T_sup_base = 0.6
T_sup_min = 0.3

Tc = np.sqrt(z0 / g)

vy_max = 0.05

In [ ]:
# basic state calculation
def calc_basic_passing_com_vel(p, T_sup):
    return (p/Tc) * ( np.sinh(T_sup/Tc) / ( 1 - np.cosh(T_sup/Tc)))    
def calc_basic_unpassing_com_vel(p, T_sup):
    return (p/Tc) * ( np.sinh(T_sup/Tc) / ( -1 - np.cosh(T_sup/Tc)))

def calc_basic_passing_foot_pos(v, T_sup):
    return Tc*v * (np.cosh(T_sup/Tc)-1) / np.sinh(T_sup/Tc)
def calc_basic_unpassing_foot_pos(v, T_sup):
    return Tc*v * (np.cosh(T_sup/Tc)+1) / np.sinh(T_sup/Tc)

In [ ]:
# calculate robot initial settings
v_mag_boarder = calc_basic_passing_com_vel(foot_dist_x_max, T_sup_base)
v_mag_max = calc_basic_passing_com_vel(foot_dist_x_max, T_sup_min)

vx_max = v_mag_max

In [ ]:
# order normalization
def normalize_vel(v, v_mag):
    if v_mag > 0.34:
        return [0.34/v_mag*v[0], 0.34/v_mag*v[1], v[2]]
    else:
        return v
    
def calc_T_sup(v_mag):
    if v_mag < 0.25:
        return 0.6
    else:
        return 0.6 - 0.3 * ( (v_mag - 0.25) / (0.34 - 0.25) )

In [ ]:
# rotation
def calc_rot_angle_limit(cvn_last):
    # return 0 for too small vel
    if abs(cvn_last[0]) < 0.005 or abs(cvn_last[1]) < 0.005:
        return [0,0]

    a1 = np.arctan2(cvn_last[1], cvn_last[0])
    y_v_lim = 0.03 / Tc
    cv_mag = np.sqrt(cvn_last[0]**2 + cvn_last[1]**2)
    if cv_mag < y_v_lim: # if no concerns to pass the pivot
        a2 = 0.5
    else:
        a2 = np.arccos(y_v_lim / np.sqrt(cvn_last[0]**2 + cvn_last[1]**2))
    a2 = a2 * -1 * (a1/abs(a1))

    # reduce angle limits
    a1*=0.3
    a2*=0.3

    angle_limits = []
    if a1 > a2:
        angle_limits = [a1, a2]
    else:
        angle_limits = [a2, a1]
    # print("angle_limits:", angle_limits)
    return angle_limits

def rotate_vec(vec, angle):
    c = np.cos(angle)
    s = np.sin(angle)
    R = np.array([[c, -s],
                  [s,  c]])
    vec_rot = R @ np.array(vec)
    return vec_rot.tolist()

In [9]:
kp = 0.01
kd = 0.001

# foot position pd
def foot_pos_pd(cvn_last, cvn_m1_last, vd, pn, T_sup):
    err = [0,0]
    err[0] = vd[0] - cvn_last[0]

    basic_foot_pos = foot_dist_y_base * -pn[1]/abs(pn[1])
    vd_y_sum = calc_basic_unpassing_com_vel(basic_foot_pos, T_sup) + vd[1]
    err[1] = vd_y_sum - cvn_last[1]

    derr = [0,0]
    derr[0] = (-cvn_last[0] + cvn_m1_last[0])
    derr[1] = (-cvn_last[1] + cvn_m1_last[1])

    p_base = [0,0]
    p_base[0] = calc_basic_passing_foot_pos(cvn_last[0], T_sup)
    p_base[1] = calc_basic_unpassing_foot_pos(cvn_last[1], T_sup)

    px = p_base[0] - kp * err[0] - kd * derr[0]
    py = p_base[1] - kp * err[1] - kd * derr[1]

    return [px, py]


In [ ]:
# LIP calculation
def calc_LIP_p(t, cp0, cv0):
    x_t = cp0[0] * np.cosh(t/Tc) + Tc * cv0[0] * np.sinh(t/Tc)
    y_t = cp0[1] * np.cosh(t/Tc) + Tc * cv0[1] * np.sinh(t/Tc)
    return [x_t, y_t]

def calc_LIP_v(t, cp0, cv0):
    vx_t = cp0[0] / Tc * np.sinh(t/Tc) + cv0[0] * np.cosh(t/Tc)
    vy_t = cp0[1] / Tc * np.sinh(t/Tc) + cv0[1] * np.cosh(t/Tc)
    return [vx_t, vy_t]

In [8]:
# double support sprine
def calc_double_support_sprine_coeff(T_ds, cpn_d_0, cvn_d_0, can_d_0, cpn_d_T, cvn_d_T, can_d_T):
    a0 = cpn_d_0
    a1 = cvn_d_0
    a2_x = 1/2*can_d_0[0]
    a2_y = 1/2*can_d_0[1]

    a3_x = 1/(2*T_ds**3) * ( 20*(cpn_d_T[0]-cpn_d_0[0]) - (8*cvn_d_T[0] + 12*cvn_d_0[0])*T_ds - (3*can_d_0[0] - can_d_T[0])*T_ds**2 )
    a3_y = 1/(2*T_ds**3) * ( 20*(cpn_d_T[1]-cpn_d_0[1]) - (8*cvn_d_T[1] + 12*cvn_d_0[1])*T_ds - (3*can_d_0[1] - can_d_T[1])*T_ds**2 )

    a4_x = 1/(2*T_ds**4) * ( 30*(cpn_d_0[0]-cpn_d_T[0]) + (14*cvn_d_T[0] + 16*cvn_d_0[0])*T_ds + (3*can_d_0[0] - 2*can_d_T[0])*T_ds**2 )
    a4_y = 1/(2*T_ds**4) * ( 30*(cpn_d_0[1]-cpn_d_T[1]) + (14*cvn_d_T[1] + 16*cvn_d_0[1])*T_ds + (3*can_d_0[1] - 2*can_d_T[1])*T_ds**2 )

    a5_x = 1/(2*T_ds**5) * ( 12*(cpn_d_T[0]-cpn_d_0[0]) - 6*(cvn_d_T[0] + cvn_d_0[0])*T_ds - (can_d_0[0] - can_d_T[0])*T_ds**2 )
    a5_y = 1/(2*T_ds**5) * ( 12*(cpn_d_T[1]-cpn_d_0[1]) - 6*(cvn_d_T[1] + cvn_d_0[1])*T_ds - (can_d_0[1] - can_d_T[1])*T_ds**2 )

    ax = [a0[0], a1[0], a2_x, a3_x, a4_x, a5_x]
    ay = [a0[1], a1[1], a2_y, a3_y, a4_y, a5_y]

    return [ax, ay]

def calc_double_support_com_p(t, coeff):
    x_t = coeff[0][0] + coeff[0][1]*t + coeff[0][2]*t**2 + coeff[0][3]*t**3 + coeff[0][4]*t**4 + coeff[0][5]*t**5
    y_t = coeff[1][0] + coeff[1][1]*t + coeff[1][2]*t**2 + coeff[1][3]*t**3 + coeff[1][4]*t**4 + coeff[1][5]*t**5

    return [x_t, y_t]

In [ ]:
def calc_z_height(t, T_sup, double_support_ratio):
    T_ss = T_sup * (1-double_support_ratio)

    return z0 + np.cos(np.pi * t / T_ss) * T_sup/20 - T_sup/4


## main code : Gait Generation

In [ ]:
# State variables

# com state at last step
cpn_last = [0,0]
cvn_last = [0,0]
cvn_m1_last = [0,0]
cvn_last = [0,0]

# foot step
pn = [0,0]
pivot = RIGHT
foot_right = [0,0,0] # from left foot. x,y,theta
foot_left = [0,0,0] # from right foot. x,y,theta

# double support calculation
cpn_d_0 = [0,0]
cvn_d_0 = [0,0]
can_d_0 = [0,0]
cpn_d_T = [0,0]
cvn_d_T = [0,0]
can_d_T = [0,0]
